# PYNQ 튜토리얼 : DMA를 스트리밍 인터페이스로 사용하기

오버레이는 2개의 DMA와 AXI스트림 FIFO(입력 및 출력 AXI 스트림 인터페이스)로 구성되어 있다. FIFO는 가속기를 나타낸다. 단일 DMA는 읽기 및 쓰기 채널이 활성화 된 상태로 사용할 수 있지만 데모 용으로 두 개의 서로 다른 DMA가 사용된다.

* 읽기 채널이 활성화 된 첫 번째 DMA는 DDR에서 IP 입력 스트림(DDR에서 읽기 및 AXI 스트림으로 전송)에서 연결된다.
* 두 번째 DMA에는 Write 채널이 활성화되어 있으며 DDR (DDR1 메모리에서 AXI 스트림을 수신하고 DDR 메모리에 쓰기)에 대한 IP 출력 스트림에 연결된다.

아래 디자인에는 실제로 다른 IP가 있으며 (현재 표시되지 않음) 아래 그림에서는 표현되지 않는다.

![](images/dma_stream_example.png)

## 1. 오버레이 다운로드
오버레이는 오버레이 클래스를 인스턴스화 할 때 자동으로 다운로드 할 수 있다.

In [1]:
from pynq import Overlay

overlay = Overlay("./bitstream/pynq_tutorial.bit")

/usr/local/lib/python3.6/dist-packages/pynq/overlay.py:299: UserWarning: Users will not get PARAMETERS / REGISTERS information through TCL files. HWH file is recommended.
  warnings.warn(message, UserWarning)


이 오버레이에서 IP를 확인할 수 있다. DMA *axi_dma_from_pl_to_ps* 및 *axi_dma_from_pl_to_ps*에 주목하라.

In [2]:
overlay.ip_dict

{'axi_dma_from_pl_to_ps': {'addr_range': 65536,
  'driver': pynq.lib.dma.DMA,
  'fullpath': 'axi_dma_from_pl_to_ps',
  'gpio': {},
  'interrupts': {'s2mm_introut': {'controller': 'system_interrupts',
    'fullpath': 'axi_dma_from_pl_to_ps/s2mm_introut',
    'index': 1}},
  'mem_id': 'SEG_axi_dma_from_pl_to_ps_Reg',
  'phys_addr': 1078001664,
  'state': None,
  'type': 'xilinx.com:ip:axi_dma:7.1'},
 'axi_dma_from_ps_to_pl': {'addr_range': 65536,
  'driver': pynq.lib.dma.DMA,
  'fullpath': 'axi_dma_from_ps_to_pl',
  'gpio': {},
  'interrupts': {'mm2s_introut': {'controller': 'system_interrupts',
    'fullpath': 'axi_dma_from_ps_to_pl/mm2s_introut',
    'index': 0}},
  'mem_id': 'SEG_axi_dma_from_ps_to_pl_Reg',
  'phys_addr': 1077936128,
  'state': None,
  'type': 'xilinx.com:ip:axi_dma:7.1'},
 'btns_gpio': {'addr_range': 65536,
  'driver': pynq.lib.axigpio.AxiGPIO,
  'fullpath': 'btns_gpio',
  'gpio': {},
  'interrupts': {'ip2intc_irpt': {'controller': 'system_interrupts',
    'fullpath'

## 2. DMA 인스턴스 생성

위에 나열된 DMA 레이블을 사용하여 두 개의 DMA 개체를 만들 수 있습니다.

In [3]:
import pynq.lib.dma

dma_send = overlay.axi_dma_from_ps_to_pl
dma_recv = overlay.axi_dma_from_pl_to_ps

/usr/local/lib/python3.6/dist-packages/pynq/lib/dma.py:200: UserWarning: Failed to find parameter c_sg_length_width; users should really use *.hwh files for overlays.
  warnings.warn(message, UserWarning)


## 3. DMA 읽기
우리는 메모리에서 일부 데이터를 읽고 다음 셀의 FIFO에 쓸 것이다.

첫 번째 단계는 연속적인 주소를 가진 메모리를 만드는 것이다. Xlnk는 메모리 버퍼를 할당하는 데 사용되고 NumPy는 버퍼 유형을 지정하는 데 사용된다.

In [4]:
from pynq import Xlnk
import numpy as np

data_size = 100

xlnk = Xlnk()

input_buffer = xlnk.cma_array(shape=(data_size,), dtype=np.uint32)

배열은 다른 NumPy 배열처럼 사용할 수 있다. 우리는 배열에 몇 가지 테스트 데이터를 쓸 수 있다. 나중에 데이터는 DMA에 의해 FIFO로 전송된다.

In [5]:
for i in range(data_size):
    input_buffer[i] = i + 0xcafe0000

배열의 내용을 확인해 보자. 다음 셀의 데이터는 PS(DDR 메모리)에서 PL(스트리밍 FIFO)로 전송된다.

In [6]:
print("Print first few values of buffer ...")
for i in range(10):
    print(hex(input_buffer[i]))

Print first few values of buffer ...
0xcafe0000
0xcafe0001
0xcafe0002
0xcafe0003
0xcafe0004
0xcafe0005
0xcafe0006
0xcafe0007
0xcafe0008
0xcafe0009


이제 DDR에서 FIFO로의 메모리 블록에서 DMA 전송을 수행 할 준비가 되었다.

In [7]:
dma_send.sendchannel.transfer(input_buffer)

## 4. DMA 쓰기
FIFO 스트림에서 다시 데이터를 읽고 MM 메모리에 쓴다. 단계는 비슷하다.

우리는 FIFO에서 데이터를 다시 읽기 전에 빈 배열을 준비 할 것이다.

In [8]:
output_buffer = xlnk.cma_array(shape=(data_size,), dtype=np.uint32)
print("Print first few values ...")
for i in range(10):
    print('0x' + format(output_buffer[i], '02x'))

Print first few values ...
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00


In [9]:
dma_recv.recvchannel.transfer(output_buffer)

다음 셀은 PL(스트리밍 FIFO)에서 PS(DDR 메모리)로 수신 된 데이터를 인쇄한다. 이것은 이전에 보낸 데이터와 동일해야 한다.

In [10]:
print("Print first few values ...")
for i in range(10):
    print('0x' + format(output_buffer[i], '02x'))

Print first few values ...
0xcafe0000
0xcafe0001
0xcafe0002
0xcafe0003
0xcafe0004
0xcafe0005
0xcafe0006
0xcafe0007
0xcafe0008
0xcafe0009


## 5. 모든 메모리 버퍼를 비우라
메모리 누수를 피하기 위해 메모리 버퍼를 비우는 것을 잊지 마라!

In [11]:
del input_buffer, output_buffer

## 부록

## AXI DMA pdf

PDF는 Jupyter 노트북에서 링크되거나 내장 될 수 있다. AXI DMA 설명서의 pdf는 아래 노트북에 내장되어 있다.

In [12]:
from IPython.display import IFrame
IFrame("pg021_axi_dma.pdf", width=800, height=800)